In [2]:
def drag_coef(U10):
    if U10<11:
        Cd = 0.0012
    elif U10>20:
        Cd = 0.0018
    else:
        Cd = (0.49+0.065*U10)*10**(-3)
        
    return Cd

In [8]:
drag_coef(21)

0.0018

## RECCAP2 datafile
Need to make sure units are correct

Follow peter for README file https://www.ncei.noaa.gov/access/ocean-carbon-data-system/oceans/

Requirements for ​README​:
● Original study citation and dataset citation if applicable
● Gas transfer velocity (​Kw​) used and the global mean thereof.
● The temperature, salinity, and wind products used to calculate ​Kw​ and​ alpha​ should
be listed.
● Handling of sea-ice with respect to air-sea CO2 fluxes.
● The procedure used to calculate ​pco2atm​ with details of the following: xCO​2​ product,
interpolation, pH​2​O correction used and pressure product.
● Other comments or idiosyncrasies in the dataset that will affect global or regional
comparison

In [2]:
import air_sea_co2_exchange as ase
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import glob
import seaflux
import pandas as pd
import datetime
from datetime import date

# Load data
This was made with the _make_files script. In the future these will be merged

In [3]:
ds_spco2 = xr.open_dataset('/local/data/artemis/workspace/gloege/ldeo-hpd/upload/LDEO-HPD_spco2_v20210425_1x1_198201-201812.nc')
ds_fgco2 = xr.open_dataset('/local/data/artemis/workspace/gloege/ldeo-hpd/upload/LDEO-HPD_fgco2_v20210425_1x1_198201-201812.nc')

ds_merge = xr.merge([ds_spco2, ds_fgco2])
ds_merge.attrs = ds_spco2.attrs

In [4]:
ds_merge

<xarray.Dataset>
Dimensions:       (lat: 180, lon: 360, model: 9, time: 408, wind: 3)
Coordinates:
  * model         (model) object 'cesm' 'cnrm' ... 'planktom' 'princeton'
  * lon           (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat           (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time          (time) datetime64[ns] 1985-01-15 1985-02-15 ... 2018-12-15
  * wind          (wind) object 'CCMP2' 'ERA5' 'JRA55'
Data variables:
    spco2         (model, time, lat, lon) float64 ...
    spco2_filled  (model, time, lat, lon) float64 ...
    fgco2         (wind, model, time, lat, lon) float64 ...
Attributes:
    institution:    Lamont-Doherty Earth Observatory (LDEO)
    version:        LDEO-HPD_v20210425
    contact:        Luke Gloege (ljg2157@columbia.edu)
    creation_date:  2021-04-25

In [5]:
#-----------------------------------------------------------------------------------------
# use fluxkit to calculate flux 
#-----------------------------------------------------------------------------------------
fluxkit_dir = '/data/artemis/observations/SOCOM/extra_files'
fluxkit_path=f'{fluxkit_dir}/FluxKit_calculation_1982-2019_v20201120.nc'
fk = (xr.open_dataset(fluxkit_path))

# change times to center on 15th of month
fk['time'] = pd.date_range(start=f'1985-01T00:00:00.000000000', 
    end=f'2018-12T00:00:00.000000000',freq='MS') + np.timedelta64(14, 'D')

# replace longitude to 0-360
fk['lon'] = list(map(lambda x: x+360 if x<0 else x, fk['lon'].values))

# sort by longitude
fk = fk.sortby('lon')

# conversions: solubility (mol/m3/uatm) and kw converted from (cm/hr) to (m/s)
cm_to_m = (1/100)
hr_to_s = (1/3600)

#-----------------------------------------------------------------------------------------
# put all the components into RECCAP2 file  
#-----------------------------------------------------------------------------------------
ds_merge['area'] = fk.area                           # m2
ds_merge['Kw'] = fk.kw_scaled * cm_to_m * hr_to_s    # m/s
ds_merge['alpha'] = fk.sol_Weiss74                   # mol/m3/uatm
ds_merge['fice'] = fk.ice.fillna(0)                  # %
ds_merge['pco2atm'] = fk.atm                         # uatm

In [6]:
ds_merge

<xarray.Dataset>
Dimensions:       (lat: 180, lon: 360, model: 9, time: 408, wind: 3)
Coordinates:
  * wind          (wind) object 'CCMP2' 'ERA5' 'JRA55'
  * model         (model) object 'cesm' 'cnrm' ... 'planktom' 'princeton'
  * lon           (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat           (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time          (time) datetime64[ns] 1985-01-15 1985-02-15 ... 2018-12-15
Data variables:
    spco2         (model, time, lat, lon) float64 ...
    spco2_filled  (model, time, lat, lon) float64 ...
    fgco2         (wind, model, time, lat, lon) float64 ...
    area          (lat, lon) float64 nan nan nan ... 1.074e+08 1.074e+08
    Kw            (wind, time, lat, lon) float64 nan nan ... 1.842e-05 1.842e-05
    alpha         (time, lat, lon) float64 nan nan nan nan ... nan nan nan nan
    fice          (time, lat, lon) float32 0.0 0.0 0.0 ... 0.9854032 0.98516124
    pco2atm       (time, lat, lon) flo

# add regional means
- first assign coordinate to dataset
- then add dataArray and make sure to assign coordiate to it. 

In [9]:
%%time
# RECCAP2 regions 
ds_regions = xr.open_dataset('/home/gloege/projects/ldeo_hpd/data/regions/RECCAP2_region_masks_all.nc')

# put regions into output file 
region_names = ['atlantic','pacific','indian','arctic','southern']
ds_merge = ds_merge.assign_coords(coords={'regions':region_names})

# calculate regional averages (mol/m2/s)
grams_in_mol = 12.01            # g/mol
sec_to_year = 86400 * 365       # sec/year
gram_to_petagram = 1 / (10**15) # Pg/g

# conversion to mol/s to Pg/yr
conversion = grams_in_mol * gram_to_petagram * sec_to_year

# calculate global fluxes
flux_tmp = ds_merge['fgco2'] * ds_merge['area'] * conversion

# masking
tmp = (flux_tmp.mean(['lat','lon']).notnull()*1)
mask = tmp.where(tmp==1)

# put region averages into list
da_list = [((flux_tmp.where(ds_regions[f'{region}']==1).sum(['lat','lon']))*mask)\
    .expand_dims('regions')\
    .assign_coords(coords={'regions':[f'{region}']}) for region in region_names]

# concenate along regions and put into merged dataset 
ds_merge['fgco2_reg'] = xr.concat(da_list, dim='regions')

flux_region = (flux_tmp.where(ds_regions[f'open_ocean']>=1).sum(['lat','lon'])) * mask
ds_merge['fgco2_glob'] = flux_region

/home/gloege/.conda/envs/tensorflow/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


CPU times: user 30.2 s, sys: 1min 10s, total: 1min 41s
Wall time: 1min 41s


In [14]:
ds_merge['fgco2_reg'].attrs = {'long_name':'regional CO2 flux density',
                       'standard_name': 'regional CO2 flux density' ,
                       'units': 'PgC/yr' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_merge['fgco2_glob'].attrs = {'long_name':'global CO2 flux density',
                       'standard_name': 'global CO2 flux density' ,
                       'units': 'PgC/yr' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_merge['Kw'].attrs = {'long_name':'Air-sea piston velocity',
                       'standard_name': 'Air-sea piston velocity' ,
                       'units': 'm/s' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

# coordinate attributes
ds_merge['time'].attrs = {'long_name':'time',}

ds_merge['lat'].attrs = {'long_name':'latitude',
                       'units': 'degrees_north' ,}

ds_merge['lon'].attrs = {'long_name':'longtude',
                       'units': 'degrees_east' ,}

ds_merge['wind'].attrs = {'long_name':'wind products',
                       'description': 'Three wind products [CCMP2, ERA5, JRA55] used for flux calculation (wnd_avg^2 + wnd_sdt^2)^0.5 ',}

ds_merge['regions'].attrs = {'long_name':'Ocean regions',
                           'description': "five ocean regions ['atlantic','pacific','indian','arctic','southern']" ,}

In [16]:
ds_merge.info()

xarray.Dataset {
dimensions:
	lat = 180 ;
	lon = 360 ;
	model = 9 ;
	regions = 5 ;
	time = 408 ;
	wind = 3 ;

variables:
	object wind(wind) ;
		wind:long_name = wind products ;
		wind:description = Three wind products [CCMP2, ERA5, JRA55] used for flux calculation (wnd_avg^2 + wnd_sdt^2)^0.5  ;
	float64 spco2(model, time, lat, lon) ;
		spco2:long_name = sea surface pCO2 ;
		spco2:standard_name = sea surface pCO2 ;
		spco2:units = uatm ;
	float64 spco2_filled(model, time, lat, lon) ;
		spco2_filled:long_name = sea surface pCO2 ;
		spco2_filled:standard_name = sea surface pCO2 ;
		spco2_filled:units = uatm ;
	object model(model) ;
		model:long_name = model name ;
		model:units = NA ;
	float64 lon(lon) ;
		lon:long_name = longtude ;
		lon:units = degrees_east ;
	float64 lat(lat) ;
		lat:long_name = latitude ;
		lat:units = degrees_north ;
	datetime64[ns] time(time) ;
		time:long_name = time ;
	float64 fgco2(wind, model, time, lat, lon) ;
		fgco2:long_name = CO2 flux density ;
		fgco2:stan

# save data

In [17]:
ds_merge.to_netcdf(f'/local/data/artemis/workspace/gloege/RECCAP2/spco2-ensemble_LDEO_HPD_1985-2018_v20210425.nc')

In [22]:
%%time
ds_mean = ds_merge.mean(['wind','model'], keep_attrs=True)

CPU times: user 12.6 s, sys: 14.1 s, total: 26.7 s
Wall time: 26.8 s


In [26]:
ds_out = ds_mean.drop(['Kw'])

In [27]:
ds_out.to_netcdf(f'/local/data/artemis/workspace/gloege/RECCAP2/spco2_LDEO_HPD_1985-2018_v20210425.nc')

# Assign attributes

In [9]:
# Variable attributes
ds_out['fice'].attrs = {'long_name':'gridcell sea ice fraction',
                       'standard_name': 'gridcell sea ice fraction' ,
                       'units': 'dimensionless [0-1]' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_out['area'].attrs = {'long_name':'grid cell area',
                       'standard_name': 'grid cell area' ,
                       'units': 'm2' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_out['Kw'].attrs = {'long_name':'Air-sea piston velocity',
                       'standard_name': 'Air-sea piston velocity' ,
                       'units': 'm/s' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_out['alpha'].attrs = {'long_name':'CO2 solubility',
                       'standard_name': 'CO2 solubility' ,
                       'units': 'mol/m3/uatm' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_out['pco2atm'].attrs = {'long_name':'atmospheric pCO2',
                       'standard_name': 'atmospheric pCO2' ,
                       'units': 'uatm' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_out['spco2_filled'].attrs = {'long_name':'sea surface pCO2, filled will climatology',
                       'standard_name': 'sea surface pCO2, filled with climatology' ,
                       'units': 'uatm' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_out['spco2'].attrs = {'long_name':'sea surface pCO2',
                       'standard_name': 'sea surface pCO2' ,
                       'units': 'uatm' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_out['fgco2'].attrs = {'long_name':'CO2 flux density',
                       'standard_name': 'CO2 flux density' ,
                       'units': 'mol/m2/s' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_out['fgco2_reg'].attrs = {'long_name':'regional CO2 flux density',
                       'standard_name': 'regional CO2 flux density' ,
                       'units': 'PgC/yr' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_out['fgco2_glob'].attrs = {'long_name':'global CO2 flux density',
                       'standard_name': 'global CO2 flux density' ,
                       'units': 'PgC/yr' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}


# coordinate attributes
ds_out['time'].attrs = {'long_name':'time',}

ds_out['lat'].attrs = {'long_name':'latitude',
                       'units': 'degrees_north' ,}

ds_out['lon'].attrs = {'long_name':'longtude',
                       'units': 'degrees_east' ,}

ds_out['wind'].attrs = {'long_name':'wind products',
                       'description': 'Three wind products used for flux calculation (wnd_avg^2 + wnd_sdt^2)^0.5',}

ds_out['regions'].attrs = {'long_name':'Ocean regions',
                           'description': "five ocean regions ['atlantic','pacific','indian','arctic','southern']" ,}


# global attributes
now = datetime.datetime.now()
ds_out.attrs = {'institution': 'Lamont-Doherty Earth Observatory (LDEO)',
                'version': f"LDEO-HPD_v{str(date.today()).replace('-','')}",
                'contact': 'Luke Gloege (ljg2157@columbia.edu)',
                'creation_date': f'{now.strftime("%Y-%m-%d")}'}


# Select data to keep

In [10]:
# reorder variables in the file
ds_out = ds_out[['spco2','spco2_filled',
    'fgco2','fgco2_glob','fgco2_reg',
    'pco2atm','Kw','alpha','fice','area']]

In [11]:
ds_out.info()

xarray.Dataset {
dimensions:
	lat = 180 ;
	lon = 360 ;
	regions = 5 ;
	time = 408 ;
	wind = 3 ;

variables:
	float64 spco2(time, lat, lon) ;
		spco2:long_name = sea surface pCO2 ;
		spco2:standard_name = sea surface pCO2 ;
		spco2:units = uatm ;
		spco2:FillValue = NaN ;
		spco2:missing_value = NaN ;
	float64 spco2_filled(time, lat, lon) ;
		spco2_filled:long_name = sea surface pCO2, filled will climatology ;
		spco2_filled:standard_name = sea surface pCO2, filled with climatology ;
		spco2_filled:units = uatm ;
		spco2_filled:FillValue = NaN ;
		spco2_filled:missing_value = NaN ;
	float64 fgco2(wind, time, lat, lon) ;
		fgco2:long_name = CO2 flux density ;
		fgco2:standard_name = CO2 flux density ;
		fgco2:units = mol/m2/s ;
		fgco2:FillValue = NaN ;
		fgco2:missing_value = NaN ;
	float64 fgco2_glob(wind, time) ;
		fgco2_glob:long_name = global CO2 flux density ;
		fgco2_glob:standard_name = global CO2 flux density ;
		fgco2_glob:units = PgC/yr ;
		fgco2_glob:FillValue = NaN ;
		fgco2

# Save data

In [13]:
ds_out.to_netcdf(f"/local/data/artemis/workspace/gloege/RECCAP2/LDEO-HPD-wind-ensemble_v{str(date.today()).replace('-','')}_1x1_198201-201812.nc")

# Average wind and save again

In [14]:
ds_tmp = ds_out.drop('Kw').mean('wind')

/home/gloege/.conda/envs/tensorflow/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [15]:
# Variable attributes
ds_tmp['fice'].attrs = {'long_name':'gridcell sea ice fraction',
                       'standard_name': 'gridcell sea ice fraction' ,
                       'units': 'dimensionless [0-1]' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_tmp['area'].attrs = {'long_name':'grid cell area',
                       'standard_name': 'grid cell area' ,
                       'units': 'm2' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_tmp['alpha'].attrs = {'long_name':'CO2 solubility',
                       'standard_name': 'CO2 solubility' ,
                       'units': 'mol/m3/uatm' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_tmp['pco2atm'].attrs = {'long_name':'atmospheric pCO2',
                       'standard_name': 'atmospheric pCO2' ,
                       'units': 'uatm' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_tmp['spco2_filled'].attrs = {'long_name':'sea surface pCO2, filled will climatology',
                       'standard_name': 'sea surface pCO2, filled with climatology' ,
                       'units': 'uatm' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_tmp['spco2'].attrs = {'long_name':'sea surface pCO2',
                       'standard_name': 'sea surface pCO2' ,
                       'units': 'uatm' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',}

ds_tmp['fgco2'].attrs = {'long_name':'CO2 flux density',
                       'standard_name': 'CO2 flux density' ,
                       'units': 'mol/m2/s' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',
                       'note':'averaged across 3 wind products'}

ds_tmp['fgco2_reg'].attrs = {'long_name':'regional CO2 flux density',
                       'standard_name': 'regional CO2 flux density' ,
                       'units': 'PgC/yr' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',
                       'note':'averaged across 3 wind products'}

ds_tmp['fgco2_glob'].attrs = {'long_name':'global CO2 flux density',
                       'standard_name': 'global CO2 flux density' ,
                       'units': 'PgC/yr' ,
                       'FillValue': 'NaN',
                       'missing_value': 'NaN',
                       'note':'averaged across 3 wind products'}


# coordinate attributes
ds_tmp['time'].attrs = {'long_name':'time',}

ds_tmp['lat'].attrs = {'long_name':'latitude',
                       'units': 'degrees_north' ,}

ds_tmp['lon'].attrs = {'long_name':'longtude',
                       'units': 'degrees_east' ,}

ds_tmp['regions'].attrs = {'long_name':'Ocean regions',
                           'description': "five ocean regions ['atlantic','pacific','indian','arctic','southern']" ,}


# global attributes
now = datetime.datetime.now()
ds_tmp.attrs = {'institution': 'Lamont-Doherty Earth Observatory (LDEO)',
                'version': f"LDEO-HPD_v{str(date.today()).replace('-','')}",
                'contact': 'Luke Gloege (ljg2157@columbia.edu)',
                'creation_date': f'{now.strftime("%Y-%m-%d")}',
                'note': 'flux calculated with three wind products and then averaged'}


In [18]:
ds_tmp.info()

xarray.Dataset {
dimensions:
	lat = 180 ;
	lon = 360 ;
	regions = 5 ;
	time = 408 ;

variables:
	float64 spco2(time, lat, lon) ;
		spco2:long_name = sea surface pCO2 ;
		spco2:standard_name = sea surface pCO2 ;
		spco2:units = uatm ;
		spco2:FillValue = NaN ;
		spco2:missing_value = NaN ;
	float64 spco2_filled(time, lat, lon) ;
		spco2_filled:long_name = sea surface pCO2, filled will climatology ;
		spco2_filled:standard_name = sea surface pCO2, filled with climatology ;
		spco2_filled:units = uatm ;
		spco2_filled:FillValue = NaN ;
		spco2_filled:missing_value = NaN ;
	float64 fgco2(time, lat, lon) ;
		fgco2:long_name = CO2 flux density ;
		fgco2:standard_name = CO2 flux density ;
		fgco2:units = mol/m2/s ;
		fgco2:FillValue = NaN ;
		fgco2:missing_value = NaN ;
		fgco2:note = averaged across 3 wind products ;
	float64 fgco2_glob(time) ;
		fgco2_glob:long_name = global CO2 flux density ;
		fgco2_glob:standard_name = global CO2 flux density ;
		fgco2_glob:units = PgC/yr ;
		fgco2_glob:

In [19]:
ds_tmp.to_netcdf(f"/local/data/artemis/workspace/gloege/RECCAP2/LDEO-HPD-average_v{str(date.today()).replace('-','')}_1x1_198201-201812.nc")

# old file

In [ ]:
    #flux_weighted = ((ds_out['fgco2'] * ds_out['area']) / ds_out['area'].sum())
    #flux_region = (flux_weighted * conversion).where(ds_regions[f'{region}']==1).sum(['lat','lon'])

In [ ]:

%%time

'''
wind_products:'NCEP1' 'NCEP2' 'CCMP2' 'ERA5' 'JRA55'

sol   : mol/m3/uatm
k2.   : m/day
spco2 : uatm
flux  : mol/m2/day
'''
    
# load pco2
# load pco2 data
data_dir = '/home/gloege/projects/ldeo_hpd/data/model_output/XGB'
ds_spco2 = xr.merge([xr.open_dataset(fl) for fl in glob.glob(f'{data_dir}/XGB*.nc')])

# GCB 2020
variables = [ 'corrected_cesm_spco2_1x1_A', 
    'corrected_csiro_spco2_1x1_A',
    'corrected_fesom_spco2_1x1_A',
    'corrected_mpi_spco2_1x1_A', 
    'corrected_cnrm_spco2_1x1_A',
    'corrected_ipsl_spco2_1x1_A',
    'corrected_planktom_spco2_1x1_A',
    'corrected_noresm_spco2_1x1_A',
    'corrected_princeton_spco2_1x1_A',]
            
        
# make dummy variable of zeros
ds_spco2['spco2'] = ds_spco2['corrected_cesm_spco2_1x1_A']*0

# add all predictions together
for var in variables:
    ds_spco2['spco2'] += ds_spco2[f'{var}']
    
# divide by number vars to average
ds_spco2['spco2'] = ds_spco2['spco2'] / len(variables)


#-----------------------------------------------------------------------------------------

fluxkit_dir = '/local/data/artemis/workspace/gloege/data/fluxkit'
fluxkit_path=f'{fluxkit_dir}/FluxKit_calculation_1982-2019_v20201026.nc'
fk = (xr.open_dataset(fluxkit_path))

# change times to center on 15th of month
fk['time'] = pd.date_range(start=f'1982-01T00:00:00.000000000', 
    end=f'2019-12T00:00:00.000000000',freq='MS') + np.timedelta64(14, 'D')

# replace longitude to 0-360
fk['lon'] = list(map(lambda x: x+360 if x<0 else x, fk['lon'].values))

# sort by longitude
fk = fk.sortby('lon')

# this isn't used
wind_product = fk.wind.values

spco2_filled = ds_spco2['spco2'].fillna(fk.spco2_clim * fk.spco2_scaling)
dpco2 = spco2_filled - fk.atm
ice_weighting = 1 - fk.ice.fillna(0)
sol = fk.sol_Weiss74
kw = fk.kw_16cmhr
#conversion = 365 # converts from mol/m2/day to mol/m2/yr
#.  (mol/m3/uatm) * (m/day) * (uatm) = mol/m2/day
flux = sol * kw * dpco2 * ice_weighting  #* conversion

# units m2
ds_spco2['area'] = fk.area
ds_spco2['fgco2'] = flux.transpose('wind','time','lat','lon') / 86400 # mol/m2/s
ds_spco2['Kw'] = fk.kw_16cmhr / 86400   # m/s
ds_spco2['alpha'] = fk.sol_Weiss74      # mol/m3/uatm
ds_spco2['fice'] = fk.ice.fillna(0)     # %
ds_spco2['pco2atm'] = fk.atm            # uatm
ds_spco2['spco2_filled'] = spco2_filled # uatm

ds_out = ds_spco2[['fgco2','spco2','spco2_filled','fice','area','Kw','pco2atm','alpha']]\
 .sel(wind=['CCMP2','ERA5','JRA55'])

In [ ]:
%%time
'''
wind_products : 'NCEP1' 'NCEP2' 'CCMP2' 'ERA5' 'JRA55'
sol   : mol/m3/uatm
k2    : cm/hr
spco2 : uatm
flux  : mol/m2/day
'''
    
#-----------------------------------------------------------------------------------------
# load pco2 data
# including xco2 
#-----------------------------------------------------------------------------------------
data_dir = '/home/gloege/projects/ldeo_hpd/data/model_output/XGB/GCB_2020_plus_xco2'
ds_spco2 = xr.merge([xr.open_dataset(fl) for fl in glob.glob(f'{data_dir}/XGB*.nc')])
ds_spco2 = ds_spco2.sel(time=slice("1985","2018"))

# GCB 2020
variables = [ 'corrected_cesm_spco2_1x1_A', 
    'corrected_csiro_spco2_1x1_A',
    'corrected_fesom_spco2_1x1_A',
    'corrected_mpi_spco2_1x1_A', 
    'corrected_cnrm_spco2_1x1_A',
    'corrected_ipsl_spco2_1x1_A',
    'corrected_planktom_spco2_1x1_A',
    'corrected_noresm_spco2_1x1_A',
    'corrected_princeton_spco2_1x1_A',]
            
        
# make dummy variable of zeros
ds_spco2['spco2'] = ds_spco2['corrected_cesm_spco2_1x1_A']*0

# add all predictions together
for var in variables:
    ds_spco2['spco2'] += ds_spco2[f'{var}']
    
# divide by number vars to average
ds_spco2['spco2'] = ds_spco2['spco2'] / len(variables)



#-----------------------------------------------------------------------------------------
# use fluxkit to calculate flux 
#-----------------------------------------------------------------------------------------
fluxkit_dir = '/data/artemis/observations/SOCOM/extra_files'
fluxkit_path=f'{fluxkit_dir}/FluxKit_calculation_1982-2019_v20201120.nc'
fk = (xr.open_dataset(fluxkit_path))

# change times to center on 15th of month
fk['time'] = pd.date_range(start=f'1985-01T00:00:00.000000000', 
    end=f'2018-12T00:00:00.000000000',freq='MS') + np.timedelta64(14, 'D')

# replace longitude to 0-360
fk['lon'] = list(map(lambda x: x+360 if x<0 else x, fk['lon'].values))

# sort by longitude
fk = fk.sortby('lon')

# fill spco2 with climatology (uatm)
spco2_filled = ds_spco2['spco2'].fillna(fk.spco2_clim * fk.spco2_scaling)

# calculate delta pco2 (uatm)
dpco2 = spco2_filled - fk.atm

# calculate ice weighting (fraction)
ice_weighting = 1 - fk.ice.fillna(0)

# solubility (mol/m3/uatm) and kw converted from (cm/hr) to (m/s)
cm_to_m = (1/100)
hr_to_s = (1/3600)
sol = fk.sol_Weiss74
kw = fk.kw_scaled * cm_to_m * hr_to_s

# calculate flux (mol/m2/s)
flux = sol * kw * dpco2 * ice_weighting 


#-----------------------------------------------------------------------------------------
# put all the components into RECCAP2 file  
#-----------------------------------------------------------------------------------------
ds_spco2['area'] = fk.area
ds_spco2['fgco2'] = flux.transpose('wind','time','lat','lon') # mol/m2/s
ds_spco2['Kw'] = fk.kw_scaled * cm_to_m * hr_to_s    # m/s
ds_spco2['alpha'] = fk.sol_Weiss74      # mol/m3/uatm
ds_spco2['fice'] = fk.ice.fillna(0)     # %
ds_spco2['pco2atm'] = fk.atm            # uatm
ds_spco2['spco2_filled'] = spco2_filled # uatm


# organize the data a little 
ds_out = ds_spco2[['fgco2','spco2','spco2_filled','fice','area','Kw','pco2atm','alpha']]\
 .sel(wind=['CCMP2','ERA5','JRA55'])

In [ ]:


# organize the data a little 
#ds_out = ds_spco2[['fgco2','spco2','spco2_filled','fice','area','Kw','pco2atm','alpha']]\
# .sel(wind=['CCMP2','ERA5','JRA55'])

In [ ]:
%%time
# calculate regional averages (mol/m2/s)
grams_in_mol = 12.01            # g/mol
sec_to_year = 86400 * 365       # sec/year
gram_to_petagram = 1 / (10**15) # Pg/g


# conversion to mol/s to Pg/yr
conversion = grams_in_mol * gram_to_petagram * sec_to_year


# calculate regional fluxes 
for region in region_names:
    print(region)
    flux_tmp = ds_merge['fgco2'] * ds_merge['area'] * conversion
    
    # masking
    tmp = (flux_tmp.mean(['lat','lon']).notnull()*1)
    mask = tmp.where(tmp==1)

    #flux_region = flux_tmp.where(ds_regions[f'{region}']==1).sum(['lat','lon']) * mask
    flux_region = (flux_tmp.where(ds_regions[f'{region}']==1).sum(['lat','lon']))*mask
    ds_merge['fgco2_reg'] = flux_region.expand_dims('regions').assign_coords(coords={'regions':[f'{region}']})

    
    
# calculate global fluxes
flux_tmp = ds_merge['fgco2'] * ds_merge['area'] * conversion

# masking
tmp = (flux_tmp.mean(['lat','lon']).notnull()*1)
mask = tmp.where(tmp==1)

flux_region = (flux_tmp.where(ds_regions[f'open_ocean']>=1).sum(['lat','lon'])) * mask
ds_merge['fgco2_glob'] = flux_region

In [ ]:
#-----------------------------------------------------------------------------------------
# use fluxkit to calculate flux 
#-----------------------------------------------------------------------------------------
fluxkit_dir = '/data/artemis/observations/SOCOM/extra_files'
fluxkit_path=f'{fluxkit_dir}/FluxKit_calculation_1982-2019_v20201120.nc'
fk = (xr.open_dataset(fluxkit_path))

# change times to center on 15th of month
fk['time'] = pd.date_range(start=f'1985-01T00:00:00.000000000', 
    end=f'2018-12T00:00:00.000000000',freq='MS') + np.timedelta64(14, 'D')

# replace longitude to 0-360
fk['lon'] = list(map(lambda x: x+360 if x<0 else x, fk['lon'].values))

# sort by longitude
fk = fk.sortby('lon')

# fill spco2 with climatology (uatm)
#spco2_filled = ds_spco2['spco2'].fillna(fk.spco2_clim * fk.spco2_scaling)

# calculate delta pco2 (uatm)
#dpco2 = spco2_filled - fk.atm

# calculate ice weighting (fraction)
ice_weighting = 1 - fk.ice.fillna(0)

# solubility (mol/m3/uatm) and kw converted from (cm/hr) to (m/s)
cm_to_m = (1/100)
hr_to_s = (1/3600)
#sol = fk.sol_Weiss74
#kw = fk.kw_scaled * cm_to_m * hr_to_s

# calculate flux (mol/m2/s)
#flux = sol * kw * dpco2 * ice_weighting 


#-----------------------------------------------------------------------------------------
# put all the components into RECCAP2 file  
#-----------------------------------------------------------------------------------------
ds_merge['area'] = fk.area
#ds_spco2['fgco2'] = flux.transpose('wind','time','lat','lon') # mol/m2/s
ds_merge['Kw'] = fk.kw_scaled * cm_to_m * hr_to_s    # m/s
ds_merge['alpha'] = fk.sol_Weiss74      # mol/m3/uatm
ds_merge['fice'] = fk.ice.fillna(0)     # %
ds_merge['pco2atm'] = fk.atm            # uatm

In [ ]:
#plt.pcolor(ds_regions['lon'], ds_regions['lat'], ds_regions['open_ocean'].where(ds_regions[f'open_ocean']>=1))
#flux_tmp = ds_merge['fgco2'] * ds_merge['area'] * conversion
#plt.plot(flux_tmp.mean(['wind','model','lat','lon']))

In [ ]:
#region='atlantic'
#flux_region = ((flux_tmp.where(ds_regions[f'{region}']==1).sum(['lat','lon']))*mask)

%%time
da_list = [((flux_tmp.where(ds_regions[f'{region}']==1).sum(['lat','lon']))*mask)\
    .expand_dims('regions')\
    .assign_coords(coords={'regions':[f'{region}']}) for region in region_names]

%%time
ds_merge['fgco2_reg'] = xr.concat(da_list, dim='regions')

# calculate global fluxes
flux_tmp = ds_merge['fgco2'] * ds_merge['area'] * conversion

# masking
tmp = (flux_tmp.mean(['lat','lon']).notnull()*1)
mask = tmp.where(tmp==1)

flux_region = (flux_tmp.where(ds_regions[f'open_ocean']>=1).sum(['lat','lon'])) * mask
ds_merge['fgco2_glob'] = flux_region